# Setup

Clone repo, install dependencies and check PyTorch and GPU.

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

%cd ..

Setup complete. Using torch 1.8.1+cu101 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)
/content


In [ ]:
%%shell
sed -i 's/nc: 80/nc: 2/g' yolov5/models/yolov5s.yaml
cat yolov5/models/yolov5s.yaml

# parameters
nc: 4  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, C3, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3

# 1. Train


Run the following code if you wanna mount your Google drive repository and load your dataset. You can uncomment others code lines if you want respectively load your dataset from generic URL or Git

In [ ]:
#Mount drive repository
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!ls drive/MyDrive

**Define variables**

In [ ]:
%env DATASET=GIT_wfilters
%env VERS=1.0
%env ZIP=GIT_wfilters.zip
%env SET=b32i640
%env ISIZE=640

env: DATASET=GIT_wfilters
env: VERS=1.0
env: ZIP=GIT_wfilters.zip
env: SET=b32i640
env: ISIZE=640


In [ ]:
%%shell
unzip -q /content/drive/MyDrive/SistemiDigitali/Datasets/"$ZIP" -d /content
echo "Done."

Done.


In [ ]:
%%shell
echo "Reduced_DT contains:"
echo "- Train images:"
ls "$DATASET"/train/images | wc -w

echo "- Test images:"
ls "$DATASET"/test/images | wc -w

echo "- Validation images:"
ls "$DATASET"/val/images | wc -w

Reduced_DT contains:
- Train images:
3396
- Test images:
724
- Validation images:
724


**Wandb**

In [ ]:
%pip install -q wandb  
import wandb
!wandb login --relogin

#For python
#wandb.init(entity='eev')
#name="dt_name"
#wandb.init(entity='eev',project="YOLOv5", name=name)

     |████████████████████████████████| 2.1MB 8.4MB/s 
     |████████████████████████████████| 133kB 35.9MB/s 
     |████████████████████████████████| 102kB 12.0MB/s 
     |████████████████████████████████| 163kB 38.2MB/s 
     |████████████████████████████████| 71kB 10.7MB/s 
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


#**File yaml for training set up**
In the following section you can create your configuration file yaml to set up the training.

Correct format for data.yaml config file

In [ ]:
# NOT RUNNABLE
train: ./dataset/train/images
val: ./dataset/valid/images
test: ./dataset/test/images

nc: 4
names: ['doors', 'handle', 'cabinet door', 'refrigerator door']

In [ ]:
%%shell
#git
echo train: /content/"$DATASET"/train/images > /content/"$DATASET"/data.yaml
echo val: /content/"$DATASET"/val/images >> /content/"$DATASET"/data.yaml
echo test: /content/"$DATASET"/test/images >> /content/"$DATASET"/data.yaml
echo ""
echo nc: 4 >> /content/"$DATASET"/data.yaml
echo names: ['doors', 'handle', 'cabinet door', 'refrigerator door'] >> /content/"$DATASET"/data.yaml

In [ ]:
%%shell
#custom
echo train: /content/"$DATASET"/train/images > /content/"$DATASET"/data.yaml
echo val: /content/"$DATASET"/val/images >> /content/"$DATASET"/data.yaml
echo test: /content/"$DATASET"/test/images >> /content/"$DATASET"/data.yaml
echo ""
echo nc: 2 >> /content/"$DATASET"/data.yaml
echo names: ['doors', 'handle'] >> /content/"$DATASET"/data.yaml

In [ ]:
!cat "$DATASET"/data.yaml

train: /content/GIT_wfilters/train/images
val: /content/GIT_wfilters/val/images
test: /content/GIT_wfilters/test/images
nc: 4
names: [doors, handle, cabinet door, refrigerator door]


Train:

In [ ]:
%%shell
#NAME=$DATASET_b32i320_settings

#redundant variables
NAME=$DATASET
NAME="$NAME"_"$SET"

cd /content/yolov5

python train.py --img "$ISIZE" --batch 32 --epochs 250 --data /content/"$DATASET"/data.yaml --weights '' --cfg models/yolov5s.yaml --cache-images --entity 'eev' --project 'YOLOv5' --name "$NAME"

github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-59-g31ee54c torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, artifact_alias='latest', batch_size=32, bbox_interval=-1, bucket='', cache_images=True, cfg='models/yolov5s.yaml', data='/content/GIT_wfilters/data.yaml', device='', entity='eev', epochs=250, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='GIT_wfilters_b32i640', noautoanchor=False, nosave=False, notest=False, project='YOLOv5', quad=False, rect=False, resume=False, save_dir='YOLOv5/GIT_wfilters_b32i640', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=32, upload_dataset=False, weights='', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir YOLOv5', view at http://localhost:6006/
2021-05-02 14:42:50.700082: I tensorflow/stream_executor/platform/defa

The weights obtained by training are saved in */content/runs/train/exp/weights/best.pt* so, we can make a backup of these results using the script under this section

In [ ]:
# it makes a copy of the results obtained during the training in '/content/drive/MyDrive/best.pt'
%%shell
NAME=$DATASET
EXT=$SET
cp /content/yolov5/YOLOv5/"$NAME"_"$EXT"/weights/best.pt /content/drive/MyDrive/SistemiDigitali/pesi/"$NAME"_"$EXT".pt

In [ ]:
%%shell
#change in data.yaml val: PATH with testset PATH
NAME=$DATASET
EXT=$SET

python yolov5/test.py --weights /content/yolov5/YOLOv5/"$NAME"_"$EXT"/weights/best.pt --data "$NAME"/data.yaml --img-size $ISIZE

Namespace(augment=False, batch_size=32, conf_thres=0.001, data='custom_dt/data.yaml', device='', exist_ok=False, img_size=320, iou_thres=0.6, name='exp', project='runs/test', save_conf=False, save_hybrid=False, save_json=False, save_txt=False, single_cls=False, task='val', verbose=False, weights=['/content/yolov5/YOLOv5/custom_dt_b32i320/weights/best.pt'])
requirements: /content/requirements.txt not found, check failed.
YOLOv5 🚀 v5.0-59-g31ee54c torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 224 layers, 7056607 parameters, 0 gradients, 16.3 GFLOPS
val: Scanning '/content/custom_dt/test/labels' images and labels... 860 found, 0 missing, 0 empty, 0 corrupted: 100% 860/860 [00:01<00:00, 727.66it/s]
val: New cache created: /content/custom_dt/test/labels.cache
               Class      Images      Labels           P           R      mAP@.5  mAP@.5:.95: 100% 27/27 [00:25<00:00,  1.04it/s]
                 all         860        1364       0.986       0.967 

# Appendix

In [ ]:
# Evolve
!python train.py --img 640 --batch 64 --epochs 100 --data coco128.yaml --weights yolov5s.pt --cache --noautoanchor --evolve